In [1]:
# Install required libraries
!pip install dash plotly pandas
import pandas as pd
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.6 MB/s eta 0:00:00


In [2]:
# --- Load Dataset ---
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_csv(file_name)


Saving Gene Expression.csv to Gene Expression.csv


In [3]:
# Data Preparation
time_points = [str(col) for col in df.columns if col != 'time']
melted_df = df.melt(id_vars='time', var_name='time_point', value_name='expression')


In [4]:
app = Dash(__name__)


In [5]:
# Define Layout
app.layout = html.Div([
    html.H1("Gene Expression Temporal Dashboard",
           style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': '30px'}),

    # Row 1: Summary Cards
    html.Div([
        html.Div([
            html.H3("Total Genes"),
            html.P(f"{len(df):,}")
        ], className='card', style={'background': '#3498db'}),

        html.Div([
            html.H3("Time Points"),
            html.P(f"{len(time_points)}")
        ], className='card', style={'background': '#2ecc71'}),

        html.Div([
            html.H3("Expression Range"),
            html.P(f"{df.iloc[:,1:].min().min():.2f} to {df.iloc[:,1:].max().max():.2f}")
        ], className='card', style={'background': '#e74c3c'}),

        html.Div([
            html.H3("Avg Variance"),
            html.P(f"{df.iloc[:,1:].var(axis=1).mean():.2f}")
        ], className='card', style={'background': '#f39c12'})
    ], className='card-container'),

    # Row 2: Gene Selector and Heatmap
    html.Div([
        dcc.Dropdown(
            id='gene-selector',
            options=[{'label': gene, 'value': gene} for gene in df['time'].unique()],
            value=df['time'].iloc[0],
            style={'width': '100%', 'marginBottom': '20px'}
        ),

        dcc.Graph(id='expression-heatmap',
                 style={'height': '400px'})
    ], style={'padding': '20px', 'border': '1px solid #ddd', 'borderRadius': '5px'}),

    # Row 3: Expression Plots
    html.Div([
        dcc.Graph(id='expression-profile'),
        dcc.Graph(id='cluster-heatmap')
    ], style={'columnCount': 2}),

    # Row 4: Statistical Analysis
    html.Div([
        dcc.Graph(figure=px.box(melted_df, x='time_point', y='expression',
                              title='Expression Distribution Across Time Points'))
    ]),

    # Data Table
    dash_table.DataTable(
        id='data-table',
        columns=[{'name': col, 'id': col} for col in df.columns],
        data=df.head(10).to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'padding': '8px',
            'whiteSpace': 'normal',
            'height': 'auto'
        }
    )
], style={'fontFamily': 'Arial', 'padding': '20px'})

In [6]:
# --- Callbacks ---
@app.callback(
    Output('expression-profile', 'figure'),
    Input('gene-selector', 'value')
)
def update_expression_profile(selected_gene):
    gene_data = df[df['time'] == selected_gene].iloc[:,1:].T.reset_index()
    gene_data.columns = ['time_point', 'expression']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=gene_data['time_point'],
        y=gene_data['expression'],
        mode='lines+markers',
        line=dict(color='#3498db', width=2),
        marker=dict(size=8)
    ))
    fig.update_layout(
        title=f'Expression Profile: {selected_gene}',
        xaxis_title='Time Point',
        yaxis_title='Expression Level',
        hovermode='x unified'
    )
    return fig

@app.callback(
    Output('expression-heatmap', 'figure'),
    Input('gene-selector', 'value')
)
def update_heatmap(selected_gene):
    sample_genes = df.sample(20) if len(df) > 20 else df
    heatmap_data = sample_genes.set_index('time').iloc[:, :].values

    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=time_points,
        y=sample_genes['time'],
        colorscale='Viridis',
        zmid=0
    ))
    fig.update_layout(
        title='Expression Heatmap (Sample Genes)',
        xaxis_title='Time Point',
        yaxis_title='Gene'
    )
    return fig

In [7]:
@app.callback(
    Output('cluster-heatmap', 'figure'),
    Input('gene-selector', 'value')
)
def update_cluster_heatmap(_):
    # Simple clustering visualization (top 50 genes for performance)
    sample = df.head(50) if len(df) > 50 else df
    fig = px.dendrogram(sample.set_index('time'),
                       color_threshold=0.5,
                       orientation='right')
    fig.update_layout(
        title='Gene Clustering Dendrogram',
        height=600
    )
    return fig


In [9]:
#Run App
app.run(mode='inline')

<IPython.core.display.Javascript object>